In [5]:
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

# Load data
data = load_diabetes()
X = data.data
y = data.target

# Define a custom scorer to use with cross_val_score (neg MSE)
def neg_mse(y_true, y_pred):
    return -mean_squared_error(y_true, y_pred)

mse_scorer = make_scorer(neg_mse, greater_is_better=True)

def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 20, 500)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    
    # Create the model
    model = xgb.XGBRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        objective='reg:squarederror',
        random_state=42,
        verbosity=0
    )
    
    # 5-fold cross-validation, scoring by negative MSE
    score = -cross_val_score(model, X, y, cv=5, scoring=mse_scorer).mean()
    
    return score  # Optuna maximizes score, so we use negative MSE

# Create study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)


[I 2025-06-03 16:24:08,071] A new study created in memory with name: no-name-907996b2-7a54-47fb-af86-ba6c34eee627
C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_validation.py:978: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_scorer.py", line 140, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\USER\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_scorer.py", line 388, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_6636\3857348816.py", line 16, in neg_mse
    return neg_mean_squared_error(y_true, y_pred)
           ^^^^^^^^^^^^^^^^^^^^^^
NameErro

In [ ]:
study.best_value

-3102.2437861038825

In [4]:
optuna.visualization.plot_parallel_coordinate(study)